# Floresta Aleatória



... DESCRIÇÃO ...

### Importar o dataset Iris

In [1]:
from sklearn import datasets

# Importa o dataset IRIS
iris = datasets.load_iris()

In [2]:
import pandas as pd

# Transforma o Dataset no formato do Pandas (biblioteca para manuseio de dados)
data = pd.DataFrame({
    'sepal length':iris.data[:,0],
    'sepal width':iris.data[:,1],
    'petal length':iris.data[:,2],
    'petal width':iris.data[:,3],
    'species':iris.target
})

# Mostra as primeiras 5 entradas
data.head()

,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


### Separar o dataset em Atributos e Classe

In [3]:
# Separar Atributos das Classes
features = data[['sepal length', 'sepal width', 'petal length', 'petal width']]
classes = data['species']

### Bagging

In [6]:
# Número de subsets
num_subsets = 10

# Número de amostras por subset
num_samples = 50

# Cria um conjunto de Subsets vazios
subsets = [[] for i in list(range(num_subsets))]

# Subamostragem aleatória com repetição
for i in range(0, num_subsets):
    subsets[i] = features.sample(n=num_samples, replace=True)
    
# Imprime as primeiras 5 entradas do subset 0
subsets[0].head()   
        

,sepal length,sepal width,petal length,petal width
49,5.0,3.3,1.4,0.2
103,6.3,2.9,5.6,1.8
121,5.6,2.8,4.9,2.0
49,5.0,3.3,1.4,0.2
85,6.0,3.4,4.5,1.6


### Subespaço Aleatório

In [12]:
import math

# Número de Atributos
num_features = math.floor(math.sqrt(features.shape[1]))
columns = list(features.head(0))

# Seleciona colunas aleatóriamente sem reposição
for i, subset in enumerate(subsets):
    subsets[i] = subset[subset.columns.to_series().sample(n=num_features)]
    
# Imprime as primeiras 5 entradas do subset 0
subsets[0].head() 

,sepal length,petal length
49,5.0,1.4
103,6.3,5.6
121,5.6,4.9
49,5.0,1.4
85,6.0,4.5


### Construir Árvores